# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** *Y*

**Names:**

* *Kristian Aurlien*
* *Mateusz Paluchowski*


---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [24]:
import pickle
import pandas as pd
import numpy as np
from utils import load_json, load_pkl

from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [2]:
sc

## Exercise 4.8: Topics extraction

Using your pre-processed courses dataset, extract topics using LDA.
    1. Print k = 10 topics extracted using LDA and give them labels. 
    2. How does it compare with LSI?
You can use the default values for all parameters.

In [35]:
# courses = load_pkl('courses.pkl')
tf_matrix = load_pkl('tf_matrix.pkl')
terms = load_pkl('terms.pkl')

In [4]:
#We want word count vectors not colums thus we need to transpose our TF_matrix 
tf_matrix_T = tf_matrix.T

In [5]:
tf_matrix_T.shape

(854, 10875)

In [29]:
#Pandas trick to format the data as in pyspark docs example
df = pd.DataFrame(data=tf_matrix_T)
df.to_csv('tf_matrix_T.csv', sep=' ', header=False, index=False)

In [30]:
data = sc.textFile("tf_matrix_T.csv")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

# Index documents with unique IDs`
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

In [51]:
ldaModel = LDA.train(corpus, k=10, optimizer='online')

In [60]:
# Learn topics
topics_baseline = ldaModel.topicsMatrix()

In [61]:
# Print top words describing topics
def describe_topics(topics, no_of_topics=10, no_of_words=10):
    for topic in range(no_of_topics):
        print("Topic " + str(topic) + ":")
        top_words_idx = np.argsort(topics[:,topic])[::-1][0:no_of_words]
        words = []
        for word_id in top_words_idx:
            words.append(terms[word_id])
        print(words)

In [62]:
describe_topics(topics_baseline)

Topic 0:
['method', 'learn', 'student', 'model', 'flow', 'cours', 'system', 'basic', 'assess', 'understand']
Topic 1:
['student', 'disadvantg', 'method', 'system', 'learn', 'begin', 'lot', 'lectur', 'usual', 'content']
Topic 2:
['train', 'rotat', 'student', 'splinkerett', 'electrokineticsdielectrophresi', 'electromagnet', 'date', 'today', 'distributor', 'webster']
Topic 3:
['method', 'student', 'fondament', 'provid', 'chemic', 'question', 'drug', 'design', 'research', 'compound']
Topic 4:
['method', 'learn', 'student', 'model', 'content', 'system', 'design', 'concept', 'basic', 'specif']
Topic 5:
['method', 'learn', 'student', 'system', 'model', 'project', 'analysi', 'present', 'design', 'teach']
Topic 6:
['student', 'optic', 'method', 'learn', 'content', 'analysi', 'chemic', 'mechan', 'topic', 'rapid']
Topic 7:
['method', 'student', 'content', 'present', 'learn', 'comput', 'nucleu', 'cours', 'remov', 'networkscharacter']
Topic 8:
['student', 'method', 'learn', 'model', 'system', 'desi

## Exercise 4.9: Dirichlet hyperparameters

1. Fix k = 10 and β = 1.01, and vary α. How does it impact the topics?

In [54]:
# LDA docConcentration must be > 1.0 (or -1 for auto) for EM 
alphas = [0.1, 1, 10, 1000] 

#### alpha=0.1

In [56]:
ldaModel_a_0_1 = LDA.train(corpus, k=10, seed=1, topicConcentration=1.01, docConcentration=0.1, optimizer='online')

In [63]:
topics_a_0_1 = ldaModel_a_0_1.topicsMatrix()

In [64]:
describe_topics(topics_a_0_1)

Topic 0:
['learn', 'student', 'comput', 'basic', 'flow', 'method', 'cours', 'session', 'model', 'sweng']
Topic 1:
['method', 'student', 'learn', 'model', 'content', 'energi', 'design', 'system', 'process', 'magnet']
Topic 2:
['train', 'rotat', 'method', 'student', 'splinkerett', 'algebra', 'electrokineticsdielectrophresi', 'electromagnet', 'learn', 'today']
Topic 3:
['method', 'student', 'fondament', 'chemic', 'provid', 'question', 'drug', 'compound', 'research', 'learn']
Topic 4:
['method', 'student', 'learn', 'model', 'design', 'content', 'concept', 'work', 'system', 'end']
Topic 5:
['student', 'method', 'learn', 'model', 'design', 'system', 'project', 'analysi', 'present', 'teach']
Topic 6:
['student', 'system', 'method', 'learn', 'analysi', 'rapid', 'model', 'structureselabor', 'ambient', 'biometr']
Topic 7:
['project', 'method', 'ic', 'nucleu', 'student', 'present', 'remov', 'networkscharacter', 'toolsaccess', 'gel']
Topic 8:
['learn', 'student', 'method', 'design', 'system', 'con

#### alpha=1

In [65]:
ldaModel_a_1 = LDA.train(corpus, k=10, seed=1, topicConcentration=1.01, docConcentration=0.1, optimizer='online')

In [66]:
topics_a_1 = ldaModel_a_1.topicsMatrix()

In [67]:
describe_topics(topics_a_1)

Topic 0:
['project', 'respons', 'sweng', 'method', 'cours', 'basic', 'learn', 'analysi', 'csr', 'ic']
Topic 1:
['disadvantg', 'statist', 'model', 'lot', 'begin', 'usual', 'energi', 'thermogravimetr', 'affect', 'nco']
Topic 2:
['train', 'rotat', 'student', 'method', 'splinkerett', 'electrokineticsdielectrophresi', 'electromagnet', 'date', 'today', 'intern']
Topic 3:
['method', 'fondament', 'chemic', 'student', 'question', 'provid', 'drug', 'compound', 'learn', 'halt']
Topic 4:
['method', 'student', 'learn', 'model', 'content', 'system', 'design', 'project', 'assess', 'concept']
Topic 5:
['method', 'fsu', 'pocessesdescrib', 'learn', 'analysi', 'student', 'organ', 'optionsperform', 'disc', 'gener']
Topic 6:
['rapid', 'topic', 'model', 'structureselabor', 'learn', 'ambient', 'content', 'group', 'keyword', 'attach']
Topic 7:
['method', 'student', 'nucleu', 'comput', 'remov', 'networkscharacter', 'present', 'managermentassess', 'harper', 'toolsaccess']
Topic 8:
['student', 'method', 'learn',

#### alpha=10

In [68]:
ldaModel_a_10 = LDA.train(corpus, k=10, seed=1, topicConcentration=1.01, docConcentration=0.1, optimizer='online')

In [69]:
topics_a_10 = ldaModel_a_10.topicsMatrix()

In [70]:
describe_topics(topics_a_10)

Topic 0:
['sweng', 'flow', 'learn', 'csr', 'futur', 'session', 'death', 'cours', 'respons', 'butterworth']
Topic 1:
['student', 'method', 'content', 'learn', 'lectur', 'model', 'mechan', 'basic', 'problem', 'energi']
Topic 2:
['train', 'rotat', 'method', 'splinkerett', 'electrokineticsdielectrophresi', 'student', 'electromagnet', 'today', 'date', 'distributor']
Topic 3:
['method', 'student', 'research', 'learn', 'keyword', 'system', 'project', 'materi', 'physic', 'halt']
Topic 4:
['method', 'student', 'learn', 'model', 'project', 'system', 'content', 'design', 'concept', 'basic']
Topic 5:
['method', 'design', 'learn', 'student', 'project', 'fsu', 'pocessesdescrib', 'gener', 'optionsperform', 'analysi']
Topic 6:
['analysi', 'rapid', 'learn', 'regul', 'structureselabor', 'ambient', 'student', 'method', 'system', 'collect']
Topic 7:
['student', 'method', 'comput', 'learn', 'content', 'cours', 'quantum', 'nucleu', 'theori', 'present']
Topic 8:
['student', 'method', 'learn', 'model', 'desig

#### alpha=1000

In [71]:
ldaModel_a_1000 = LDA.train(corpus, k=10, seed=1, topicConcentration=1.01, docConcentration=0.1, optimizer='online')

In [72]:
topics_a_1000 = ldaModel_a_1000.topicsMatrix()

In [73]:
describe_topics(topics_a_1000)

Topic 0:
['sweng', 'csr', 'understand', 'respons', 'comput', 'death', 'futur', 'flow', 'butterworth', 'glassermanmathemat']
Topic 1:
['student', 'learn', 'method', 'model', 'content', 'lectur', 'group', 'polici', 'statist', 'problem']
Topic 2:
['train', 'rotat', 'student', 'algebra', 'splinkerett', 'electrokineticsdielectrophresi', 'electromagnet', 'today', 'date', 'method']
Topic 3:
['method', 'student', 'structur', 'research', 'learn', 'system', 'physic', 'design', 'analysi', 'keyword']
Topic 4:
['student', 'method', 'learn', 'model', 'system', 'content', 'design', 'concept', 'work', 'project']
Topic 5:
['method', 'student', 'learn', 'project', 'present', 'organ', 'system', 'teach', 'design', 'fsu']
Topic 6:
['method', 'learn', 'student', 'system', 'content', 'analysi', 'design', 'assess', 'keyword', 'mechan']
Topic 7:
['student', 'method', 'present', 'content', 'nucleu', 'learn', 'comput', 'project', 'remov', 'cours']
Topic 8:
['learn', 'student', 'method', 'model', 'design', 'syste

# TODO How does it impact the topics?


_______


2. Fix k=10 and α=6, and vary β. How does it impact the topics?

In [79]:
betas = [0.1, 1, 10, 1000] 

#### beta = 0.1

In [81]:
ldaModel_b_0_1 = LDA.train(corpus, k=10, seed=1, topicConcentration=0.1, docConcentration=6.0, optimizer='online')

In [82]:
topics_b_0_1 = ldaModel_b_0_1.topicsMatrix()

In [83]:
describe_topics(topics_b_0_1)

Topic 0:
['learn', 'method', 'cours', 'student', 'model', 'basic', 'analysi', 'system', 'comput', 'flow']
Topic 1:
['student', 'content', 'learn', 'method', 'model', 'lectur', 'statist', 'energi', 'basic', 'design']
Topic 2:
['student', 'method', 'learn', 'model', 'design', 'project', 'system', 'cours', 'outcom', 'electromagnet']
Topic 3:
['method', 'student', 'system', 'keyword', 'design', 'learn', 'research', 'introduct', 'physic', 'expect']
Topic 4:
['method', 'learn', 'student', 'concept', 'transvers', 'content', 'model', 'design', 'specif', 'system']
Topic 5:
['method', 'student', 'learn', 'design', 'project', 'system', 'model', 'analysi', 'teach', 'present']
Topic 6:
['student', 'learn', 'model', 'method', 'content', 'analysi', 'system', 'chemic', 'design', 'outcom']
Topic 7:
['method', 'student', 'content', 'learn', 'present', 'cours', 'comput', 'end', 'system', 'exercis']
Topic 8:
['student', 'learn', 'method', 'comput', 'content', 'system', 'applic', 'design', 'linear', 'exerc

#### beta = 1

In [84]:
ldaModel_b_1 = LDA.train(corpus, k=10, seed=1, topicConcentration=1.00, docConcentration=6.0, optimizer='online')

In [85]:
topics_b_1 = ldaModel_b_1.topicsMatrix()

In [86]:
describe_topics(topics_b_1)

Topic 0:
['learn', 'method', 'student', 'cours', 'model', 'basic', 'analysi', 'system', 'comput', 'flow']
Topic 1:
['student', 'method', 'learn', 'content', 'model', 'lectur', 'statist', 'design', 'system', 'basic']
Topic 2:
['student', 'method', 'learn', 'model', 'design', 'project', 'system', 'cours', 'outcom', 'electromagnet']
Topic 3:
['method', 'student', 'system', 'learn', 'keyword', 'design', 'research', 'model', 'introduct', 'halt']
Topic 4:
['method', 'learn', 'student', 'concept', 'content', 'transvers', 'model', 'design', 'system', 'specif']
Topic 5:
['method', 'student', 'learn', 'design', 'system', 'model', 'project', 'analysi', 'teach', 'present']
Topic 6:
['student', 'learn', 'method', 'model', 'content', 'analysi', 'system', 'design', 'chemic', 'assess']
Topic 7:
['method', 'student', 'learn', 'content', 'present', 'cours', 'comput', 'end', 'system', 'prerequisit']
Topic 8:
['student', 'learn', 'method', 'comput', 'content', 'system', 'design', 'applic', 'linear', 'exer

#### beta = 10

In [87]:
ldaModel_b_10 = LDA.train(corpus, k=10, seed=1, topicConcentration=10.0, docConcentration=6.0, optimizer='online')

In [88]:
topics_b_10 = ldaModel_b_10.topicsMatrix()

In [89]:
describe_topics(topics_b_10)

Topic 0:
['method', 'learn', 'student', 'model', 'cours', 'basic', 'system', 'analysi', 'comput', 'flow']
Topic 1:
['student', 'method', 'learn', 'content', 'model', 'lectur', 'system', 'design', 'statist', 'disadvantg']
Topic 2:
['student', 'method', 'learn', 'model', 'design', 'system', 'project', 'content', 'cours', 'splinkerett']
Topic 3:
['method', 'student', 'learn', 'system', 'design', 'keyword', 'model', 'research', 'content', 'halt']
Topic 4:
['method', 'learn', 'student', 'model', 'content', 'concept', 'system', 'design', 'transvers', 'specif']
Topic 5:
['method', 'student', 'learn', 'design', 'model', 'system', 'project', 'analysi', 'present', 'prerequisit']
Topic 6:
['student', 'method', 'learn', 'model', 'content', 'system', 'analysi', 'design', 'chemic', 'assess']
Topic 7:
['method', 'student', 'learn', 'content', 'present', 'cours', 'system', 'comput', 'model', 'end']
Topic 8:
['student', 'learn', 'method', 'content', 'system', 'comput', 'design', 'model', 'applic', 'exe

#### beta = 1000

In [90]:
ldaModel_b_1000 = LDA.train(corpus, k=10, seed=1, topicConcentration=1000.0, docConcentration=6.0, optimizer='online')

In [91]:
topics_b_1000 = ldaModel_b_1000.topicsMatrix()

In [92]:
describe_topics(topics_b_1000)

Topic 0:
['method', 'learn', 'student', 'model', 'cours', 'system', 'basic', 'analysi', 'comput', 'flow']
Topic 1:
['student', 'method', 'learn', 'content', 'model', 'lectur', 'system', 'design', 'statist', 'disadvantg']
Topic 2:
['student', 'method', 'learn', 'model', 'design', 'system', 'content', 'project', 'cours', 'splinkerett']
Topic 3:
['method', 'student', 'learn', 'system', 'design', 'model', 'keyword', 'content', 'research', 'halt']
Topic 4:
['method', 'learn', 'student', 'model', 'content', 'system', 'concept', 'design', 'transvers', 'specif']
Topic 5:
['method', 'student', 'learn', 'design', 'model', 'system', 'project', 'analysi', 'present', 'prerequisit']
Topic 6:
['student', 'method', 'learn', 'model', 'content', 'system', 'analysi', 'design', 'chemic', 'assess']
Topic 7:
['method', 'student', 'learn', 'content', 'system', 'present', 'cours', 'comput', 'model', 'end']
Topic 8:
['student', 'learn', 'method', 'content', 'system', 'comput', 'design', 'model', 'applic', 'exe

## Exercise 4.10: EPFL's taught subjects

For the symmetric distribution, a high alpha-value means that each document is likely to contain a mixture of most of the topics, and not any single topic specifically. A low alpha value puts less such constraints on documents and means that it is more likely that a document may contain mixture of just a few, or even only one, of the topics. Likewise, a high beta-value means that each topic is likely to contain a mixture of most of the words, and not any word specifically, while a low value means that a topic may contain a mixture of just a few of the words.

In [95]:
# Number of sections at EPFL
potential_k = len(['Architecture, Civil and Environmental Engineering ENAC',
'Basic Sciences SB',
'Engineering STI',
'Computer and Communication Sciences IC',
'Life Sciences SV',
'Management of Technology CDM',
'College of Humanities CDH'])

In [102]:
potential_alpha = 0.5
potential_beta = 15.0
potential_k

7

In [103]:
ldaModel_EPFL = LDA.train(corpus,
                          k=potential_k,
                          seed=1,
                          docConcentration=potential_alpha,
                          topicConcentration=potential_beta,
                          optimizer='online')

In [104]:
topics_EPFL = ldaModel_EPFL.topicsMatrix()

In [105]:
describe_topics(topics_EPFL, potential_k)

Topic 0:
['method', 'student', 'learn', 'model', 'cours', 'system', 'basic', 'analysi', 'content', 'project']
Topic 1:
['student', 'method', 'learn', 'content', 'model', 'system', 'design', 'lectur', 'basic', 'outcom']
Topic 2:
['student', 'method', 'learn', 'model', 'design', 'system', 'content', 'project', 'cours', 'outcom']
Topic 3:
['method', 'student', 'learn', 'system', 'design', 'model', 'content', 'keyword', 'research', 'assess']
Topic 4:
['method', 'learn', 'student', 'model', 'content', 'system', 'design', 'concept', 'project', 'transvers']
Topic 5:
['method', 'student', 'learn', 'design', 'model', 'system', 'project', 'analysi', 'present', 'prerequisit']
Topic 6:
['student', 'method', 'learn', 'model', 'content', 'system', 'design', 'analysi', 'assess', 'cours']


## Exercise 4.11: Wikipedia structure